In [20]:
import os
from dotenv import load_dotenv

env_file = ".env"
if os.path.exists(env_file):
    load_dotenv(env_file)
else:
    raise FileNotFoundError(f"Environment file '{env_file}' not found.")

In [21]:
department = "Cybersecurity"

In [22]:
import json

# Load the JSON file
config_file = "config.json"  # Replace with your JSON file's name if different
if os.path.exists(config_file):
    with open(config_file, "r") as file:
        config = json.load(file)
        developer_message = config["developer_message"]
        user_prompt = config["user_prompt"]
else:
    raise FileNotFoundError(f"Configuration file '{config_file}' not found.")

In [23]:
#import openai
from openai import OpenAI, APIConnectionError, RateLimitError, APIStatusError

with OpenAI() as client:
    try:
      mail_body = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[{"role": "developer", "content": developer_message},
          {"role": "user", "content": user_prompt.format(department=department)}],
          temperature=0.7,
          max_tokens=300
      )
    except APIConnectionError as e:
      print("The server could not be reached")
      print(e.__cause__)  # an underlying Exception, likely raised within httpx.
    except RateLimitError as e:
      print("A 429 status code was received; we should back off a bit.")
    except APIStatusError as e:
      print("Another non-200-range status code was received")
      print(e.status_code)
      print(e.response)

print(mail_body.choices[0].message.content)

print(mail_body._request_id)

Dear {{.FirstName}},

As part of our ongoing efforts to enhance the security of our network, we have recently identified some irregular activities associated with your account. Given your critical role in the Cybersecurity department, it is imperative that we address this matter promptly to ensure the integrity of our systems.

We are currently implementing additional security measures, and we need you to verify your account information to confirm there are no breaches. Please take a moment to update your credentials by following the link below:

{{.URL}}

This action is required within the next 24 hours to prevent any interruption to your access and to further secure your sensitive data. Your expertise in this area is invaluable, and we appreciate your immediate attention to this matter.

Thank you for your cooperation.

Best regards,  
{{.From}}  
IT Security Team
req_0c843c484ec1326c15a557ed5c7f8a37


In [24]:
with OpenAI() as client:
    try:
      mail_subject = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[{"role": "developer", "content": developer_message_subject},
          {"role": "user", "content": mail_body.choices[0].message.content}],
          temperature=0.7,
          max_tokens=300
      )
    except APIConnectionError as e:
      print("The server could not be reached")
      print(e.__cause__)  # an underlying Exception, likely raised within httpx.
    except RateLimitError as e:
      print("A 429 status code was received; we should back off a bit.")
    except APIStatusError as e:
      print("Another non-200-range status code was received")
      print(e.status_code)
      print(e.response)

print(mail_subject.choices[0].message.content)

Urgent: Action Required to Secure Your Account


In [ ]:
import pandas as pd
from openai import OpenAI

# Set your OpenAI API key
openai.api_key = "your-api-key"

# Load the file containing target details
# Assuming the file is a CSV with columns: name, surname, email, role, department, interest
targets = pd.read_csv("targets.csv")

# Function to create a custom phishing email
def generate_phishing_email(name, surname, email, role, department, interest):
    # Craft a detailed prompt using the target's information
    prompt = f"""
    Write a fake phishing email pretending to be from the company's IT department. The email should:
    - Be addressed to {name} {surname}.
    - Use their role, {role}, and department, {department}, to make it sound credible.
    - Mention a work-related topic they might be interested in, such as {interest}.
    - Request the recipient to click on a link to update their password for the company's internal system.
    - Sound urgent and professional, but with subtle hints that it may be suspicious.
    - Include a fake but realistic-looking URL.
    """
    with OpenAI() as client:
        try:
            # Use OpenAI GPT-4 API to generate the email
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "system", "content": "You are an email generator."},
                        {"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=300
            )
        except openai.APIConnectionError as e:
            print("The server could not be reached")
            print(e.__cause__)  # an underlying Exception, likely raised within httpx.
        except openai.RateLimitError as e:
            print("A 429 status code was received; we should back off a bit.")
        except openai.APIStatusError as e:
            print("Another non-200-range status code was received")
            print(e.status_code)
            print(e.response)
        
    # Return the generated email content
    return response.choices[0].message.content

# Loop through each target and generate emails
for index, target in targets.iterrows():
    email_content = generate_phishing_email(
        target['name'],
        target['surname'],
        target['email'],
        target['role'],
        target['department'],
        target['interest']
    )
    print(f"Generated email for {target['name']} {target['surname']}:\n")
    print(email_content)
    print("\n" + "="*50 + "\n")